In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("9 covid_toy.csv")
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [3]:
df["city"].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

# Train Test Split

In [4]:
from sklearn.model_selection import train_test_split

In [30]:
X = df.drop("has_covid",axis = 1)
y = df["has_covid"]

X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.2 , random_state = 42)

In [8]:
X_train.shape , X_test.shape

((80, 5), (20, 5))

In [11]:
y_train

55    Yes
88     No
26    Yes
42    Yes
69     No
     ... 
60    Yes
71     No
14     No
92     No
51    Yes
Name: has_covid, Length: 80, dtype: object

# Without column transformer

In [9]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [13]:
#Fill missing values.

from sklearn.impute import SimpleImputer

imputer = SimpleImputer()

X_train_fever = imputer.fit_transform(X_train[["fever"]])
X_test_fever = imputer.fit_transform(X_test[["fever"]])

X_train_fever.shape,X_test_fever.shape

((80, 1), (20, 1))

In [14]:
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


In [15]:
df["cough"].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [18]:
#Encoding categorical values

from sklearn.preprocessing import OrdinalEncoder

ordinal = OrdinalEncoder(categories=[["Mild","Strong"]])

X_train_cough = ordinal.fit_transform(X_train[["cough"]])
X_test_cough = ordinal.fit_transform(X_test[["cough"]])

X_train_cough.shape , X_test_cough.shape

((80, 1), (20, 1))

In [20]:
#Gender and city encoding

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(drop="first",sparse_output=False)

X_train_gender_city = ohe.fit_transform(X_train[["gender","city"]])
X_test_gender_city = ohe.fit_transform(X_test[["gender","city"]])

X_train_gender_city.shape

(80, 4)

In [21]:
# Extracting age in another variable

X_train_age = X_train.drop(columns=["gender","fever","cough","city"]).values
X_test_age = X_test.drop(columns=["gender","fever","cough","city"]).values

X_train_age.shape , X_test_age.shape

((80, 1), (20, 1))

In [23]:
X_train_transform = np.concatenate((X_train_age , X_train_fever ,X_train_gender_city , X_train_cough),axis=1)
X_test_transform = np.concatenate((X_test_age , X_test_fever , X_test_gender_city , X_test_cough),axis = 1)

X_train_transform.shape

(80, 7)

# With Column Transformer

In [29]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [26]:
from sklearn.compose import ColumnTransformer

In [33]:
tf = ColumnTransformer(
    transformers=[
        ("tf1",SimpleImputer(),["fever"]),
        ("tf2",OrdinalEncoder(categories=[["Mild","Strong"]]),["cough"]),
        ("tf3",OneHotEncoder(drop="first",sparse_output=False),["gender","city"])
        
    ],
    remainder = "passthrough"
)

In [34]:
tf

,transformers,"[('tf1', ...), ('tf2', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'mean'
,fill_value,None


In [35]:
X_train_tf = tf.fit_transform(X_train)
X_test_tf = tf.fit_transform(X_test)

In [36]:
X_train_tf

array([[101.,   0.,   0.,   0.,   0.,   1.,  81.],
       [100.,   0.,   0.,   0.,   1.,   0.,   5.],
       [100.,   0.,   0.,   0.,   1.,   0.,  19.],
       [100.,   0.,   1.,   1.,   0.,   0.,  27.],
       [103.,   0.,   0.,   1.,   0.,   0.,  73.],
       [103.,   1.,   1.,   0.,   1.,   0.,  70.],
       [102.,   0.,   0.,   1.,   0.,   0.,  49.],
       [101.,   1.,   0.,   0.,   1.,   0.,  51.],
       [101.,   0.,   0.,   1.,   0.,   0.,  64.],
       [101.,   0.,   0.,   0.,   1.,   0.,  83.],
       [ 98.,   0.,   0.,   0.,   0.,   1.,  65.],
       [104.,   0.,   0.,   0.,   0.,   0.,  18.],
       [103.,   0.,   0.,   0.,   0.,   0.,  16.],
       [104.,   0.,   1.,   0.,   1.,   0.,  16.],
       [100.,   0.,   1.,   0.,   1.,   0.,  27.],
       [101.,   0.,   0.,   0.,   0.,   0.,  84.],
       [104.,   0.,   1.,   0.,   1.,   0.,  51.],
       [102.,   0.,   0.,   0.,   0.,   0.,  69.],
       [102.,   1.,   0.,   0.,   0.,   0.,  82.],
       [103.,   0.,   0.,   0.,